In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import chi2_contingency
from scipy.stats import zscore
from scipy.stats import linregress
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [37]:
# PRÉ PROCESSAMENTO

In [38]:
dados_clientes = pd.read_csv("C:/Projetos Pessoais/DataScience/segmentacao_Clientes_Ecomerce/data/dados_clientes_para_pre_processing.csv")

In [39]:
dados_clientes

,Cod_cliente,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,diversidade_produtos,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
0,12347,2,182,4310.00,23.68,103,1,0,615.714286,341.070789,4.486071
1,12348,75,27,1437.24,53.23,21,3,0,359.310000,203.875689,-100.884000
2,12349,18,72,1457.55,20.24,72,0,0,1457.550000,0.000000,0.000000
3,12350,310,16,294.40,18.40,16,2,0,294.400000,0.000000,0.000000
4,12352,36,77,1385.74,18.00,57,1,0,346.435000,120.042154,-2.682000
...,...,...,...,...,...,...,...,...,...,...,...
5505,21821,1,2,3.35,1.68,2,3,1,3.350000,0.000000,0.000000
5506,21822,1,634,5699.00,8.99,634,3,1,5699.000000,0.000000,0.000000
5507,21823,0,730,6756.06,9.25,730,4,1,6756.060000,0.000000,0.000000
5508,21824,0,59,3217.20,54.53,56,4,1,3217.200000,0.000000,0.000000


# Padronização das variáveis

##### Não irei noramlizar as seguintes variáveis:

- Cod_cliente: Pois se refere a código de cada cliente, sem valor expressivo ou relevante;

- UK: Pois já está no formato booleano, 0 ou 1;

- Dia_fav_compra: Pois ´variável categórica e ja está normalizada (1 a 7)


In [40]:
# Variáveis excluidas
var_exclude = ['Cod_cliente', 'Dia_fav_compra', 'UK']

In [41]:
# Inicando o objeto Scaler 
scaler = StandardScaler()

dados = dados_clientes.copy()
colunas_padronizadas = dados.columns.difference(var_exclude)
colunas_padronizadas

Index(['Avg_transacao', 'Desvio_padrao_gastos_mensais',
       'Gastos_medios_mensais', 'Tendencia_gastos', 'Total_transacoes',
       'Ultima_Compra', 'Valor_total_gasto', 'diversidade_produtos'],
      dtype='object')

In [42]:
# Treinando a padronização nos dados - exceto variáveis listadas em var_exclude
dados[colunas_padronizadas] = scaler.fit_transform(dados[colunas_padronizadas])

In [43]:
# Amostra do dataframe já padronizado
dados.sample(8)

,Cod_cliente,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,diversidade_produtos,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
880,13555,-0.380942,0.375300,1.362487,0.037881,0.652541,1,1,1.570021,3.069327,6.025302
1336,14199,0.924808,-0.655807,-0.554056,0.680646,-0.731797,3,1,-0.415880,-0.510682,0.035669
5105,20720,0.195389,-0.371099,-0.513592,-0.206140,-0.358246,0,1,-0.327908,-0.510682,0.035669
860,13521,-1.029314,2.676053,-0.115525,-0.251513,2.674113,3,1,-0.036625,1.397674,3.228496
4589,19532,1.573181,0.944717,0.032103,-0.208620,1.520498,1,1,0.858467,-0.510682,0.035669
4070,17990,1.086901,-0.448047,-0.338685,-0.052658,-0.457127,6,1,-0.279208,-0.506397,0.028500
4828,20165,0.915803,0.567670,0.007212,-0.191050,1.004118,4,1,0.804353,-0.510682,0.035669
1103,13871,-0.840205,1.914265,2.452285,-0.080460,0.949184,3,1,0.062321,0.750285,0.165386


In [44]:
# Indexação com código cliente
dados.set_index('Cod_cliente', inplace = True)

In [53]:
dados

,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,diversidade_produtos,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
Cod_cliente,,,,,,,,,,
12347,-1.020309,0.729262,1.440195,-0.032710,0.377870,1,0,0.036451,1.271037,0.063393
12348,-0.362931,-0.463437,0.051211,0.272704,-0.523048,3,0,-0.233072,0.554344,-0.587805
12349,-0.876226,-0.117169,0.061030,-0.068264,0.037279,0,0,0.921358,-0.510682,0.035669
12350,1.753284,-0.548080,-0.501355,-0.087281,-0.577982,2,0,-0.301303,-0.510682,0.035669
12352,-0.714133,-0.078695,0.026310,-0.091416,-0.127523,1,0,-0.246606,0.116406,0.019094
...,...,...,...,...,...,...,...,...,...,...
21821,-1.029314,-0.655807,-0.642078,-0.260091,-0.731797,3,1,-0.607244,-0.510682,0.035669
21822,-1.029314,4.207324,2.111779,-0.184538,6.211865,3,1,5.379817,-0.510682,0.035669
21823,-1.038319,4.946027,2.622869,-0.181851,7.266599,4,1,6.490961,-0.510682,0.035669


In [51]:
dados_clientes

,Cod_cliente,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,diversidade_produtos,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
0,12347,2,182,4310.00,23.68,103,1,0,615.714286,341.070789,4.486071
1,12348,75,27,1437.24,53.23,21,3,0,359.310000,203.875689,-100.884000
2,12349,18,72,1457.55,20.24,72,0,0,1457.550000,0.000000,0.000000
3,12350,310,16,294.40,18.40,16,2,0,294.400000,0.000000,0.000000
4,12352,36,77,1385.74,18.00,57,1,0,346.435000,120.042154,-2.682000
...,...,...,...,...,...,...,...,...,...,...,...
5505,21821,1,2,3.35,1.68,2,3,1,3.350000,0.000000,0.000000
5506,21822,1,634,5699.00,8.99,634,3,1,5699.000000,0.000000,0.000000
5507,21823,0,730,6756.06,9.25,730,4,1,6756.060000,0.000000,0.000000
5508,21824,0,59,3217.20,54.53,56,4,1,3217.200000,0.000000,0.000000


In [45]:
dados

,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,diversidade_produtos,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
Cod_cliente,,,,,,,,,,
12347,-1.020309,0.729262,1.440195,-0.032710,0.377870,1,0,0.036451,1.271037,0.063393
12348,-0.362931,-0.463437,0.051211,0.272704,-0.523048,3,0,-0.233072,0.554344,-0.587805
12349,-0.876226,-0.117169,0.061030,-0.068264,0.037279,0,0,0.921358,-0.510682,0.035669
12350,1.753284,-0.548080,-0.501355,-0.087281,-0.577982,2,0,-0.301303,-0.510682,0.035669
12352,-0.714133,-0.078695,0.026310,-0.091416,-0.127523,1,0,-0.246606,0.116406,0.019094
...,...,...,...,...,...,...,...,...,...,...
21821,-1.029314,-0.655807,-0.642078,-0.260091,-0.731797,3,1,-0.607244,-0.510682,0.035669
21822,-1.029314,4.207324,2.111779,-0.184538,6.211865,3,1,5.379817,-0.510682,0.035669
21823,-1.038319,4.946027,2.622869,-0.181851,7.266599,4,1,6.490961,-0.510682,0.035669


In [48]:
# Salvando dados para clusterizar
dados.to_csv('../data/dados_processados.csv', sep =',', index = True, encoding = 'utf-8')

In [52]:
# Salvando DF não processado para futura avaliação
dados_clientes.to_csv('../data/dados_para_avaliacao_final.csv', sep =',', index = True, encoding = 'utf-8')

In [21]:
dados_clientes

,Cod_cliente,Ultima_Compra,Total_transacoes,Valor_total_gasto,Avg_transacao,Dia_fav_compra,UK,Gastos_medios_mensais,Desvio_padrao_gastos_mensais,Tendencia_gastos
0,12347,2,182,4310.00,23.68,1,0,615.714286,341.070789,4.486071
1,12348,75,27,1437.24,53.23,3,0,359.310000,203.875689,-100.884000
2,12349,18,72,1457.55,20.24,0,0,1457.550000,0.000000,0.000000
3,12350,310,16,294.40,18.40,2,0,294.400000,0.000000,0.000000
4,12352,36,77,1385.74,18.00,1,0,346.435000,120.042154,-2.682000
...,...,...,...,...,...,...,...,...,...,...
5505,21821,1,2,3.35,1.68,3,1,3.350000,0.000000,0.000000
5506,21822,1,634,5699.00,8.99,3,1,5699.000000,0.000000,0.000000
5507,21823,0,730,6756.06,9.25,4,1,6756.060000,0.000000,0.000000
5508,21824,0,59,3217.20,54.53,4,1,3217.200000,0.000000,0.000000
